In [26]:
%pylab inline
np.set_printoptions(precision=2, suppress=True)
np.__version__

Populating the interactive namespace from numpy and matplotlib


'1.10.1'

In [18]:
def new_positions_spherical_coordinates(num_points, radius, x_offset=0.0, y_offset=0.0, z_offset=0.0, stddev=0.0):
    radius = radius + stddev * np.random.rand()
    print(radius)
    theta = np.random.uniform(0.,2. * np.pi ,(num_points,1))
    phi = np.arccos(1-2*np.random.uniform(0.0,1.,(num_points,1)))
    x = radius * np.sin( theta ) * np.cos( phi ) + x_offset + stddev * np.random.rand()
    y = radius * np.sin( theta ) * np.sin( phi ) + y_offset + stddev * np.random.rand()
    z = radius * np.cos( theta ) + z_offset + stddev * np.random.rand()
    return (x,y,z)

In [48]:
# Generate random point set
r = 20.0
n = 100
(x,y,z) = new_positions_spherical_coordinates(n, r,
                                              x_offset=1.0,
                                              y_offset=2.0,
                                              z_offset=3.0, 
                                              stddev=0.01)
# print(x)
# print(y)
# print(z)

20.0094789505


In [49]:
# Point set from Hildenbrand
# n = 5
# x = [1.0, 1.0, 0.0, 0.0, -1.0]
# y = [0.0, 1.0, 0.0, 1.0,  0.0]
# z = [0.0, 0.0, 1.0, 1.0,  1.0]

In [50]:
ps = [np.array(np.ravel(p).tolist() + 
               [1.0, 0.5*np.linalg.norm(p)**2]).reshape(5,1) 
      for p in zip(x,y,z)]

## Total Least Squares Fitting of $k$-Spheres in $n$-D Euclidean Space Using an $(n + 2)$-D Isometric Representation

Leo Dorst 2014

Journal of Mathematical Imaging and Vision

http://link.springer.com/article/10.1007%2Fs10851-014-0495-2


Cost function:
$$\frac{1}{N}\sum_i\frac{(p_i \cdot x)^2}{x^2}$$

In [51]:
M = np.zeros((5,5))
M[:3,:3] = np.eye(3)
M[3,4] = -1
M[4,3] = -1
P = np.zeros((5,5)) 
for p in ps:
    P += np.outer(p,p)
PM = np.matmul(P,M)
(w,v) = np.linalg.eig(PM)
idx = np.where(w == np.min(w[w>0]))[0][0]
x_ = v[:,idx]
x_ /= x_[3]
x_dorst = x_.copy()
print(x_)
radius = np.sqrt(x_[0]*x_[0] + x_[1]*x_[1] + x_[2]*x_[2] - 2*x_[4])
print("Estimated radius: {}".format(radius))

[   1.      2.      3.      1.   -193.17]
Estimated radius: 20.0094789505


In [52]:
print PM

[[    6688.52      624.81      917.63   -37643.83     -139.35]
 [     624.81    13771.22      266.57   -78467.21     -256.1 ]
 [     917.63      266.57    21809.54  -141861.72     -387.89]
 [     139.35      256.1       387.89   -21134.64     -100.  ]
 [   37643.83    78467.21   141861.72 -4703471.78   -21134.64]]


In [53]:
(u,s,v) = np.linalg.svd(PM) 
x_ = v.T[:,-1]
x_ /= x_[3]
print(x_)
radius = np.sqrt(x_[0]*x_[0] + x_[1]*x_[1] + x_[2]*x_[2] - 2*x_[4])
print("Estimated radius: {}".format(radius))

[   1.      2.      3.      1.   -193.17]
Estimated radius: 20.0094789505


## Foundations of Geometric Algebra Computing

Dietmar Hildenbrand 2013

In [58]:
P = np.zeros((5,5))
for p in ps:
    P += np.outer(p,p)
(w,v) = np.linalg.eig(P)
idx = np.where(w == np.min(w[w>0]))[0][0]
x_ = v[:,idx]
x_ /= x_[4]
print(x_)
radius = np.sqrt(x_[0]*x_[0] + x_[1]*x_[1] + x_[2]*x_[2] - 2*x_[3])
print("Estimated radius: {}".format(radius))

[-114.22   -0.9    -2.4     0.65    1.  ]
Estimated radius: 114.238427445


In [ ]:
# P

In [ ]:
import sys
sys.path.append('../build/Debug/')
import libsphere_fit as sphere_fit

In [ ]:
def create_solver_options(use_trust_region_minimizer=True):
    solver_options = {
        'minimizer_type':'TRUST_REGION',
#                 'minimizer_type':'LINE_SEARCH',
                      'trust_region_strategy_type':'LEVENBERG_MARQUARDT',
                      'linear_solver_type':'DENSE_QR',
                      'max_num_iterations': 100, 
                      'num_threads': 12,
                      'num_linear_solver_threads':12,
                      'parameter_tolerance': 10e-8,
                      'function_tolerance': 10e-8,
                      'minimizer_progress_to_stdout':False,
                      'trust_region_minimizer_iterations_to_dump':[],
                      'trust_region_problem_dump_directory':'/home/lars/devel/game_ws/dump/sphere_fit',
                     }
    return solver_options    

In [59]:
sf = sphere_fit.SphereFit()

solver_options = create_solver_options()
# solver_options['minimizer_progress_to_stdout'] = True
# solver_options['trust_region_minimizer_iterations_to_dump'] = [1,2,3,4,5,6,7,8,9]
sf.set_solver_options(solver_options)


sphere = np.array([1.0, 1.0, 1.0, 1.0, 1.0]).reshape(5,1)
points = np.ascontiguousarray(np.transpose([x,y,z])).reshape(n,3).copy()
x_ = sf.run(sphere, points).copy()

x_ = x_ / x_[3]
print(x_.reshape(-1))
radius = np.sqrt(x_[0]*x_[0] + x_[1]*x_[1] + x_[2]*x_[2] - 2*x_[4])[0]
print("Estimated radius: {}".format(radius))

NameError: name 'sphere_fit' is not defined

In [ ]:
np.linalg.norm(sphere)
x_dorst

In [ ]:
# matplotlib.rc?
scale = 1.0
fig_width_pt = 358.50475  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (sqrt(5.0)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height =fig_width*golden_mean       # height in inches
fig_size = [fig_width,fig_height]
print fig_size
params = {
    'axes.labelsize': 9,
    'font.size': 9,
#           "font.family": "sans-serif",
#           "font.sans-serif": [],
          "font.family": "serif",
          "font.serif": [],
          'legend.fontsize': 9,
          'legend.linewidth': 0.3,
          'xtick.labelsize': 8,
          'ytick.labelsize': 8,
          'text.usetex': True,
          'figure.figsize': fig_size,
          'axes.linewidth': 0.5
         }
 
 
matplotlib.rcParams.update(params)
iterations = sf.summary()['iterations']
plt.plot([iteration['gradient_max_norm'] for iteration in iterations])

plt.xlabel('Iteration $k$')
plt.ylabel('Gradient max norm')
plt.xticks([iteration['iteration'] for iteration in iterations])
save_dir = '/home/lars/Dropbox/sharelatex/2016_phd_thesis_lt/figures'
plt.savefig(save_dir + '/sphere_fit_gradient.pdf')

In [ ]:
sphere.reshape(-1)

In [ ]:
print(sf.summary()['full_report'])

In [ ]:
"a\\r\\nb"

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_aspect("equal")
u, v = np.mgrid[0:2*np.pi:20j, 0:-np.pi:10j]
sx= r * np.cos(u)*np.sin(v)
sy= r * np.sin(u)*np.sin(v)
sz= r * np.cos(v)
ax.plot_wireframe(sx, sy, sz, color="r")

ax.scatter3D(x,y,z)
# sx= radius * np.cos(u)*np.sin(v) + x_[0]
# sy= radius * np.sin(u)*np.sin(v) + x_[1]
# sz= radius * np.cos(v) + x_[2]
# ax.plot_wireframe(sx, sy, sz, color="b")
plt.savefig('sphere_fit.pdf')

In [ ]:
def load_problem_iteration_dump(directory, iteration):
    tmp = '/ceres_solver_iteration_{:0>3d}'.format(iteration)
    D = np.loadtxt(directory + tmp + '_D.txt')
    b = np.loadtxt(directory + tmp + '_b.txt')
    x = np.loadtxt(directory + tmp + '_x.txt')
    f = open(directory + tmp + '.m')
    matlab_script = f.readlines()
    f.close()
    rows = [int(s) for s in matlab_script[1][:-2].split() if s.isdigit()][0]
    cols = [int(s) for s in matlab_script[2][:-2].split() if s.isdigit()][0]
    A_sparse = np.loadtxt(directory + tmp + '_A.txt')
    A = np.zeros((rows, cols))
    for row in A_sparse:
        A[int(row[0]), int(row[1])] = row[2]
    return (A,D,b,x)

directory = solver_options['trust_region_problem_dump_directory']
(A, D, b, x_) = load_problem_iteration_dump(directory, 1)
A

In [ ]:
D